In [1]:
import pandas as pd
from google.cloud import bigquery
import os
from google.oauth2 import service_account

In [2]:
# Global inputs
corrections_standort = [
    {"duisburg": ["duisubrg", "duis", "duisburgburg"]},
    {"hamburg": ["hamburg harburg", "hamburg norderstedt", "hh hamburg", "hh harburg"]},
    {"zurich": ["zürich", "dfp zurich"]}
]

In [3]:
# If the path does not contain csv_dateien, then the current working directory will be add it to the path
if "csv_dateien" not in os.getcwd():
    os.chdir(os.getcwd() + "/csv_dateien")

In [4]:
# List all CSV files in the current directory starting with the word "spielerbeurteilung"
csv_files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith("spielerbeurteilung") and f.endswith(".csv")]
print(csv_files)
# Import the dataset
df_list = []
for filename in csv_files:
    df_iter = pd.read_csv(filename, encoding="unicode_escape")
    # Extract the year from the file name and create a new column called year showing the year of the dataset
    df_iter["year"] = "20" + str(filename.split("_")[2].split(".")[0])
    df_list.append(df_iter)

df = pd.concat(df_list)

['spielerbeurteilung_aug_23.csv', 'spielerbeurteilung_nov_23.csv', 'spielerbeurteilung_okt_23.csv', 'spielerbeurteilung_dec_23.csv']


In [5]:
# Make the names of all columns lowercase
df.columns = map(str.lower, df.columns)

In [6]:
# Replace all spaces in column names with underscores and remove all special characters
special_chars = ["(", ")", "/", "-", ",", "_", "."]
for char in special_chars:
    df.columns = df.columns.str.replace(char, "")

# Replace the umlauts with the corresponding letters
umlauts = {"ä": "ae", "ö": "oe", "ü": "ue", "ß": "ss"}
for umlaut in umlauts:
    df.columns = df.columns.str.replace(umlaut, umlauts[umlaut])

# Strip extra spaces at the end of the column names
df.columns = df.columns.str.strip()

# Replace the double spaces with single spaces
df.columns = df.columns.str.replace("  ", " ")

# Replace all spaces with underscores
df.columns = df.columns.str.replace(" ", "_")

df.columns

Index(['monat', 'name_spieler', 'trainer', 'standort', 'ballfuehrung',
       'finten', 'dribbling', '2_gegen_2_offensiv', '2_gegen_2_defensiv',
       'torschuss', 'antrittsschnelligkeit', 'grundschnelligkeit',
       'laufbewegungsgeschicklichkeit', 'aktionsschnelligkeit',
       'spielintelligenz_kreativitaet', 'durchsetzungsvermoegen',
       'willensstaerke', 'konzentrationsvermoegen', 'selbstvertrauen',
       'kritikfaehigkeit', 'bemerkungen_und_hinweise', 'year',
       'jonglieren_mit_rueckdrall_links_und_rechts',
       'passspiel_innenseite_rechts', 'passspiel_innenseite_links',
       'ballmitnahme_erster_kontakt_nach_vorne_rechte_innenseite',
       'ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite',
       'ballmitnahme_erster_kontakt_90_grad_rechte_innenseite',
       'ballmitnahme_erster_kontakt_90_grad_linke_innenseite',
       'ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_rechte_innenseite',
       'ballmitnahme_erster_kontakt_mit_offener_stel

In [7]:
# Eliminate the NULL months
df = df[df["monat"].notnull()]

# Strip the extra spaces at the end of the month names
df["monat"] = df["monat"].str.lower().str.strip()

# Take the first three letters of any month
df["monat"] = df["monat"].str[:3]
df["monat"].unique()

array(['aug', 'nov', 'okt', 'dec'], dtype=object)

In [8]:
# Fill all "-" values with None
df = df.replace("-", None)

# Strip the extra spaces at the end of the trainer and standort names
for col in ["trainer", "standort"]:
    df[col] = df[col].str.lower().str.strip()

# Change any occurences of [duis, duisburg, or duisubrg] to duisburg
for corr in corrections_standort:
    for item in list(corr.values())[0]:
        df["standort"] = df["standort"].str.replace(item, list(corr.keys())[0])

# Melt the dataset so that all columns after standort excluding bemerkungen_und_hinweise are in one column
df_melted = pd.melt(
    df,
    id_vars=["year", "monat", "name_spieler", "trainer", "standort", "bemerkungen_und_hinweise"],
    var_name="kriterium",
    value_name="bewertung",
)

# Change the data type of the bewertung column to float
df_melted["bewertung"] = df_melted["bewertung"].astype(float)
df_melted

,year,monat,name_spieler,trainer,standort,bemerkungen_und_hinweise,kriterium,bewertung
0,2023,aug,Roland,johan,zurich,Keine,ballfuehrung,7.0
1,2023,aug,Ljen,johan,zurich,"Wir müssen weiter arbeiten, vor allem körperli...",ballfuehrung,7.0
2,2023,aug,Julien,johan,zurich,Julien muss seine Mentalität verbessern. Was d...,ballfuehrung,7.0
3,2023,aug,Selim,johan,zurich,Er muss sich noch mehr anstrengen.\r\nDie Verl...,ballfuehrung,7.0
4,2023,aug,Bujar,johan,zurich,"Er hat sich durch die Kritik sehr verbessert, ...",ballfuehrung,7.0
...,...,...,...,...,...,...,...,...
15115,2023,dec,Leonardo Polo Friz,sergio,duisburg,NaN,disziplin,NaN
15116,2023,dec,Maximilian Raabe,sergio,duisburg,NaN,disziplin,NaN
15117,2023,dec,James Michael Rusch,sergio,duisburg,NaN,disziplin,NaN
15118,2023,dec,Ben Steinbring,sergio,duisburg,NaN,disziplin,NaN


In [9]:
# Upload the dataset to BigQuery
key_path_home_dir = os.path.expanduser("~") + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path_home_dir, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
bq_client = bigquery.Client(project="web-scraping-371310", credentials=credentials)

# Create the job configuration of df_booked_appts
job_config_fussball_akademie = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("year", "STRING"),
        bigquery.SchemaField("monat", "STRING"),
        bigquery.SchemaField("name_spieler", "STRING"),
        bigquery.SchemaField("trainer", "STRING"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("bemerkungen_und_hinweise", "STRING"),
        bigquery.SchemaField("kriterium", "STRING"),
        bigquery.SchemaField("bewertung", "FLOAT64"),
    ]
)
job_config_fussball_akademie.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bq_client.load_table_from_dataframe(
    dataframe=df_melted,
    destination="web-scraping-371310.crawled_datasets.spielerbeurteilung",
    job_config=job_config_fussball_akademie
).result()

LoadJob<project=web-scraping-371310, location=EU, id=f3e14a94-4366-4051-8892-1eee2235396b>

In [10]:
# Select six columns from "df"
df_radar_view_test = df[[
    # ID columns
    "year", "monat", "name_spieler", "trainer", "standort", "bemerkungen_und_hinweise",
    
    # Numeric columns
    "jonglieren_mit_rueckdrall_links_und_rechts",
    "passspiel_innenseite_rechts", "passspiel_innenseite_links",
    "ballmitnahme_erster_kontakt_nach_vorne_rechte_innenseite",
    "ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite",
    "ballmitnahme_erster_kontakt_90_grad_rechte_innenseite",
    "ballmitnahme_erster_kontakt_90_grad_linke_innenseite",
    "ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_rechte_innenseite",
    "ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_linke_innenseite",
    "ballmitnahme_erster_kontakt_90_grad_rechte_aussenseite",
    "ballmitnahme_erster_kontakt_90_grad_linke_aussenseite",
    "ballmitnahme_erster_kontakt_180_grad_rechte_aussenseite",
    "ballmitnahme_erster_kontakt_180_grad_linke_aussenseite",
    "jonglieren_mit_rueckdrall",
    "ballhaendling_innenseite_vollspann_oberschenkel_brust_liftenvolley",
    "enge_ballfuehrung_lr", "tempodribbling_lire", "koerperfinte_lire",
    "uebersteiger_lire", "torschuss_innenseite_lire",
    "torschuss_vollspann_lire", "2_gegen_2_offensiv", "2_gegen_2_defensiv",
    "antrittsschnelligkeit", "grundschnelligkeit",
    "laufbewegungsgeschicklichkeit",
    "aktionsschnelligkeit_bewegung_mit_ball",
    "spielintelligenz_kreativitaet", "durchsetzungsvermoegen_willenstaerke",
    "konzentrationsvermoegen", "selbstvertrauen", "kritikfaehigkeit",
    "disziplin", "ballfuehrung", "finten", "dribbling", "torschuss",
    "aktionsschnelligkeit", "durchsetzungsvermoegen", "willensstaerke"
]]

# Change all the data types of the numeric columns to float
df_radar_view_test.loc[:, "jonglieren_mit_rueckdrall_links_und_rechts":] = df_radar_view_test.loc[:, "jonglieren_mit_rueckdrall_links_und_rechts":].astype(float)

# Create the job configuration of df_booked_appts
job_config_radar_test = bigquery.LoadJobConfig(
    schema = [
        # ID columns
        bigquery.SchemaField("year", "STRING"),
        bigquery.SchemaField("monat", "STRING"),
        bigquery.SchemaField("name_spieler", "STRING"),
        bigquery.SchemaField("trainer", "STRING"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("bemerkungen_und_hinweise", "STRING"),

        # Numeric columns
        bigquery.SchemaField("jonglieren_mit_rueckdrall_links_und_rechts", "FLOAT64"),
        bigquery.SchemaField("passspiel_innenseite_rechts", "FLOAT64"),
        bigquery.SchemaField("passspiel_innenseite_links", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_nach_vorne_rechte_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_rechte_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_linke_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_rechte_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_linke_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_rechte_aussenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_linke_aussenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_180_grad_rechte_aussenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_180_grad_linke_aussenseite", "FLOAT64"),
        bigquery.SchemaField("jonglieren_mit_rueckdrall", "FLOAT64"),
        bigquery.SchemaField("ballhaendling_innenseite_vollspann_oberschenkel_brust_liftenvolley", "FLOAT64"),
        bigquery.SchemaField("enge_ballfuehrung_lr", "FLOAT64"),
        bigquery.SchemaField("tempodribbling_lire", "FLOAT64"),
        bigquery.SchemaField("koerperfinte_lire", "FLOAT64"),
        bigquery.SchemaField("uebersteiger_lire", "FLOAT64"),
        bigquery.SchemaField("torschuss_innenseite_lire", "FLOAT64"),
        bigquery.SchemaField("torschuss_vollspann_lire", "FLOAT64"),
        bigquery.SchemaField("2_gegen_2_offensiv", "FLOAT64"),
        bigquery.SchemaField("2_gegen_2_defensiv", "FLOAT64"),
        bigquery.SchemaField("antrittsschnelligkeit", "FLOAT64"),
        bigquery.SchemaField("grundschnelligkeit", "FLOAT64"),
        bigquery.SchemaField("laufbewegungsgeschicklichkeit", "FLOAT64"),
        bigquery.SchemaField("aktionsschnelligkeit_bewegung_mit_ball", "FLOAT64"),
        bigquery.SchemaField("spielintelligenz_kreativitaet", "FLOAT64"),
        bigquery.SchemaField("durchsetzungsvermoegen_willenstaerke", "FLOAT64"),
        bigquery.SchemaField("konzentrationsvermoegen", "FLOAT64"),
        bigquery.SchemaField("selbstvertrauen", "FLOAT64"),
        bigquery.SchemaField("kritikfaehigkeit", "FLOAT64"),
        bigquery.SchemaField("disziplin", "FLOAT64"),
        bigquery.SchemaField("ballfuehrung", "FLOAT64"),
        bigquery.SchemaField("finten", "FLOAT64"),
        bigquery.SchemaField("dribbling", "FLOAT64"),
        bigquery.SchemaField("torschuss", "FLOAT64"),
        bigquery.SchemaField("aktionsschnelligkeit", "FLOAT64"),
        bigquery.SchemaField("durchsetzungsvermoegen", "FLOAT64"),
        bigquery.SchemaField("willensstaerke", "FLOAT64"),

    ]
)
job_config_radar_test.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bq_client.load_table_from_dataframe(
    dataframe=df_radar_view_test,
    destination="web-scraping-371310.crawled_datasets.spielerbeurteilung_radar",
    job_config=job_config_radar_test
).result()

LoadJob<project=web-scraping-371310, location=EU, id=a6eae294-dabb-4f59-8cff-d6dbb469536f>